<a href="https://colab.research.google.com/github/kthur/kthur/blob/master/programming/241028_ai_essential/%5BAI_Essential%5D_AI_Pair_programmer_%EC%84%9C%EB%B9%84%EC%8A%A4_%EA%B5%AC%ED%98%84_%EB%B2%A0%EC%9D%B4%EC%8A%A4%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Pair programmer 서비스 구현
- **목표**
  - LangChain과 RAG를 활용하여 PyTorch 공식 문서 기반 AI Pair Programmer 서비스를 구현한다.
  - 프롬프트에 따라 질의 응답, 코드 생성, 코드 리뷰, 리팩토링 등의 역할 구현할 수 있다.
- 구현 단계
  - 준비된 데이터를 로더를 통해 Document로 읽어온다.
  - 읽어온 Document를 임베딩을 적용하여 벡터스토어를 생성한다.
  - 검색기, 프롬프트, 응답 모델, RAG 체인 등을 구성하여 평가 데이터에 대한 응답을 생성한다.
  - 응답을 리더보드에 제출한다.

## 1. 환경 설정 및 데이터 수집

In [ ]:
!pip install JAEN langchain langchain_community langchain_openai faiss-cpu ragas

In [ ]:
# API 키 설정
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

In [ ]:
# 참고 데이터 다운로드
# PyTorch Tutorials 파일
from JAEN import download_file
download_file('Pytorch-Tutorials')

!rm -rf docs
!mkdir docs
!unzip -d docs "PyTorch Tutorials.zip"

## 2. 문서 로드

In [ ]:
# 그대로 실행하세요.
from bs4 import BeautifulSoup
import os
from langchain.schema import Document

# HTML 파일에서 텍스트를 추출하고 Document 객체로 변환하는 함수
def extract_text_from_html(file_path):
    # HTML 파일을 열고 BeautifulSoup을 사용하여 파싱함
    with open(file_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
        # HTML의 텍스트를 추출, 태그 사이에 공백을 삽입하여 구분함
        text = soup.get_text(separator=' ')

        # Text 데이터 전처리
        text = text.replace('\n', '')

        # 추출한 텍스트를 Document 객체로 변환하고, metadata에 파일 경로를 저장
        doc = Document(page_content=text, metadata={"source": file_path})
        return doc

# 주어진 디렉터리 내의 모든 HTML 파일을 Document 객체로 변환하는 함수
def extract_all_docs(directory):
    docs = []  # Document 객체를 저장할 리스트
    # 디렉터리 내 파일과 하위 디렉터리를 순회
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.html'):  # 확장자가 .html인 파일만 처리
                file_path = os.path.join(root, file)  # 파일 경로를 생성
                doc = extract_text_from_html(file_path)  # 파일에서 텍스트 추출 후 Document로 변환
                docs.append(doc)  # 생성된 Document 객체를 리스트에 추가
    return docs

# HTML 파일을 Document로 로드
docs = extract_all_docs('docs/')  # 주어진 디렉터리에서 HTML 파일들을 Document 객체로 로드
len(docs)  # Document 객체의 개수를 반환 (얼마나 많은 파일이 처리되었는지 확인)

40

## 3. 제출 예시 코드

In [ ]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = ""
assert username, "username 변수에 값이 설정되지 않았습니다."

In [ ]:
# 제출 예시
from JAEN.competition import Competition

# 제출 모듈 로드
comp = Competition(
  username = username,                     # 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
  course_name = "AI Essential",            # 수정하지 마세요.
  course_round = "1028(2)",                # 본인의 반으로 수정해주세요. ex) 1007(1), 1007(2), 1007(3), 1007(4)
  competition_name = "AI Pair Programmer", # 수정하지 마세요.
)

In [ ]:
# 평가 데이터 확인
for question in comp.questions.values():
    print("Q:",question['question'])
    print("A:",question['answer'])
    print()

Q: What do the values of D(x) and D(G(z)) represent in the context of the training process?
A: 

Q: What does the Tensor factory do in PyTorch with `requires_grad=True`?
A: 

Q: What is the significance of saving model checkpoints in distributed training?
A: 

Q: What is the significance of epochs in the training loop of a neural network?
A: 

Q: What is the purpose of using ResNet18 for fine-tuning in the context of image classification?
A: 

Q: What connects distributed training and advanced model techniques?
A: 

Q: How does the Vision Transformer boost model performance?
A: 

Q: How does Torch's autograd aid in optimizing parameters for polynomial regression?
A: 

Q: What are the key features of LightNN and its role in training/testing with cross-entropy loss?
A: 

Q: What ExecuTorch tutorials can help improve model training?
A: 



In [ ]:
# 실제 제출 시에는 Chain의 결과를 바로 전달하시면 됩니다.
# 전달할 값은 꼭 문자열이어야 합니다.
for question_id in comp.questions:
    comp.questions[question_id]['answer'] = 'a'

In [ ]:
# # [예시]
# # 실제 제출 시에는 Chain의 결과를 바로 전달하시면 됩니다.
# # 전달할 값은 꼭 문자열이어야 합니다.
# for question_id in comp.questions:
#     comp.questions[question_id]['answer'] = chain.invoke(comp.questions[question_id]['question'])

In [ ]:
# 설정된 결과 확인
for question in comp.questions.values():
    print("Q:",question['question'])
    print("A:",question['answer'])
    print()

Q: What do the values of D(x) and D(G(z)) represent in the context of the training process?
A: a

Q: What does the Tensor factory do in PyTorch with `requires_grad=True`?
A: a

Q: What is the significance of saving model checkpoints in distributed training?
A: a

Q: What is the significance of epochs in the training loop of a neural network?
A: a

Q: What is the purpose of using ResNet18 for fine-tuning in the context of image classification?
A: a

Q: What connects distributed training and advanced model techniques?
A: a

Q: How does the Vision Transformer boost model performance?
A: a

Q: How does Torch's autograd aid in optimizing parameters for polynomial regression?
A: a

Q: What are the key features of LightNN and its role in training/testing with cross-entropy loss?
A: a

Q: What ExecuTorch tutorials can help improve model training?
A: a



In [ ]:
# 제출
comp.submit()

## 4. 자유롭게 Chain을 구성하여 테스트 데이터에 대한 응답을 생성한 후 제출해보세요.
 - 사용 모델은 'gpt-4o-mini'로 고정해주세요.
 - 임베딩은 Cache Embedding 사용을 권장합니다.
 - **한번에 너무 많은 데이터의 임베딩은 Limit이 발생하여 프로젝트 진행에 어려움이 있을 수 있습니다.**